# IBM Coursera Capstone Project

## 1.Business Problem: Subway Business Opportunities
     - Select few POIs of subway, which locate in differenct areas of Beijing, China.
     - Analyze venue categories, count and distribuation around each POI.
     - Use machine learning algorithm, e.g. Kmeans, to execute Venue's analysis
     - Provide meaningful businese suggestions based on above model's output(predication), businese type, locations and so on. e.g. It's better to open "hotpot" restaurant in Daxing district, but it's better to open "convenience store" in Sihui area.
     - Besides of venue information, there are other important factors which affect final business decision as below, but they are out of scope of this project.
       -> Price of House of area
       -> If there is subway station of area
       -> population density of area
     
##  2. DataSet:
     - Select few POIs from Baidu map
     - Get longitude and latitude of each POIs based on coordinate system of Baidu.
     - Save POI's name, longitude and latitude in .csv file as dataset 
     - Retrieve venue information of each POIs by using Fourqure APIs

## 3. Project implemenation 

### 3.1 Import All Libraries and Packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
beijing_df = pd.read_csv('/root/Notebooks/coursera/coursera_capstone_project/Geospatial_Beijing_Coordinates.csv')

In [3]:
beijing_df.head(30)

,Subway Station,Longitude,Latitude
0,GuoMao Subway Station,116.468148,39.914832
1,Sihui East Subway Station,116.522091,39.914846
2,Tongzhou Subway Station,116.672144,39.903694
3,Jintai Road Subway Station,116.484865,39.929399
4,TuanJieHu Subway Station,116.468363,39.939510
5,WangJing Subway Station,116.474634,40.004785
6,WangJing East Subway Station,116.493202,40.009192
7,Jiangtai Subway Station,116.496723,39.976800
8,Guozhan Subway Station,116.561760,40.075863
9,Lishuiqiao South Subway Station,116.420987,40.048115


In [4]:
beijing_df.shape

(29, 3)

### 3.2 get the latitude,longitude of Beijing

In [5]:
address = 'Beijing,China'
geolocator = Nominatim(user_agent="bj_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Beijing are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Beijing are 39.9020803, 116.7185213.


### 3.3 Create map of Beijing by using folium

In [7]:
map_beijing = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(beijing_df['Latitude'], beijing_df['Longitude'],beijing_df['Subway Station']):
    label = '{}, {}'.format(neighborhood, 'Beijing')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_beijing)  
    
map_beijing 

### 3.4 Explore the subway stations by using Foursquare APIs

In [8]:
CLIENT_ID = 'PVBTYHASBMH1NGZUITEIGVZO3FUTBPBL1WSYMA010JUN0QXV' # your Foursquare ID
CLIENT_SECRET = '3ZNVZWN4Y4C31WHVD2KOUNVQWJPGSNSR2QJU1XY2KY51W4AQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PVBTYHASBMH1NGZUITEIGVZO3FUTBPBL1WSYMA010JUN0QXV
CLIENT_SECRET:3ZNVZWN4Y4C31WHVD2KOUNVQWJPGSNSR2QJU1XY2KY51W4AQ


In [9]:
# 3 kilometers, and 100 venues
radius = 3000
LIMIT = 100

In [10]:

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Subway Station', 
                  'Subway Latitude', 
                  'Subway Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
# type your answer here
beijing_venues = getNearbyVenues(names=beijing_df['Subway Station'],
                                   latitudes=beijing_df['Latitude'],
                                   longitudes=beijing_df['Longitude']
                                  )

GuoMao Subway Station
Sihui East Subway Station
Tongzhou Subway Station
Jintai Road Subway Station
TuanJieHu Subway Station
WangJing Subway Station
WangJing East Subway Station
Jiangtai Subway Station
Guozhan Subway Station
Lishuiqiao South Subway Station
Tiantongyuan Subway Station
Xierqi Subway Station
Life Science Park Subway Station
Shahe Subway Station
Changping Subway Station
Wukesong Subway Station
Gucheng Subway Station
Wuluju Subway Station
Tiancun Subway Station
Fengtai Subway Station
Liangxiang Subway Station
Changyang Subway Station
Liangxiang university city Subway Station
Suzhuang Subway Station
Fangshan Subway Station
Caoqiao Subway Station
Daxing Airport Subway Station
Yizhuang Bridge Subway Station
Yizhuang Railway Station Subway Station


In [12]:
print(beijing_venues.shape)
beijing_venues.head()

(725, 7)


,Subway Station,Subway Latitude,Wubway Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,GuoMao Subway Station,39.914832,116.468148,Din Tai Fung (鼎泰丰),39.909210,116.473768,Dumpling Restaurant
1,GuoMao Subway Station,39.914832,116.468148,Annie's 安妮意大利餐厅,39.912178,116.470545,Italian Restaurant
2,GuoMao Subway Station,39.914832,116.468148,Rosewood Beijing (北京瑰丽酒店),39.918612,116.454916,Hotel
3,GuoMao Subway Station,39.914832,116.468148,Hotel Jen Beijing (新国贸饭店),39.911456,116.453576,Hotel
4,GuoMao Subway Station,39.914832,116.468148,Apple China Central Mall (Apple 华贸购物中心),39.909304,116.479679,Electronics Store


In [13]:
beijing_venues.groupby('Subway Station').count()

,Subway Latitude,Wubway Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Subway Station,,,,,,
Caoqiao Subway Station,39,39,39,39,39,39
Changping Subway Station,7,7,7,7,7,7
Changyang Subway Station,5,5,5,5,5,5
Daxing Airport Subway Station,5,5,5,5,5,5
Fengtai Subway Station,4,4,4,4,4,4
Gucheng Subway Station,10,10,10,10,10,10
GuoMao Subway Station,100,100,100,100,100,100
Guozhan Subway Station,29,29,29,29,29,29
Jiangtai Subway Station,72,72,72,72,72,72


### How many unique venue category

In [29]:
print('There are {} uniques categories.'.format(len(beijing_venues['Venue Category'].unique())))

There are 130 uniques categories.


### List all categories

In [32]:
beijing_venues['Venue Category'].unique()[:]

array(['Dumpling Restaurant', 'Italian Restaurant', 'Hotel',
       'Electronics Store', 'Peking Duck Restaurant', 'Hotpot Restaurant',
       'Department Store', 'Gym / Fitness Center', 'Beijing Restaurant',
       'French Restaurant', 'Bar', 'Shopping Mall', 'Sushi Restaurant',
       'Gastropub', 'Café', 'Vietnamese Restaurant', 'Hotel Bar',
       'New American Restaurant', 'Coffee Shop', 'Chinese Restaurant',
       'Theater', 'Art Gallery', 'Pizza Place', 'Massage Studio',
       'Skating Rink', 'Thai Restaurant', 'Multiplex',
       'Spanish Restaurant', 'Cantonese Restaurant', 'Dessert Shop',
       'Supermarket', 'Tea Room', 'Japanese Restaurant', 'Cocktail Bar',
       'Bakery', 'Hainan Restaurant', 'Restaurant', 'Whisky Bar',
       'Soup Place', 'Comfort Food Restaurant', 'Xinjiang Restaurant',
       'Mexican Restaurant', 'Park', 'Asian Restaurant', 'Sandwich Place',
       'Clothing Store', 'Farm', 'Antique Shop', 'Bookstore',
       'Dim Sum Restaurant', 'Fast Food Resta

In [37]:
    "Hotpot Restaurant" in beijing_venues['Venue Category'].unique()

True

### Filiter all "hotpot" restaurants

In [47]:
beijing_hotpot_venues = beijing_venues[beijing_venues['Venue Category'] == 'Hotpot Restaurant']
print(beijing_hotpot_venues.shape)
beijing_hotpot_venues.head(20)

(12, 7)


,Subway Station,Subway Latitude,Wubway Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,GuoMao Subway Station,39.914832,116.468148,Haidilao Hot Pot (海底捞火锅),39.914400,116.476463,Hotpot Restaurant
96,GuoMao Subway Station,39.914832,116.468148,羊大爷涮肉,39.921710,116.480619,Hotpot Restaurant
117,Sihui East Subway Station,39.914846,116.522091,Nanmen Hotpot (南门涮肉),39.915552,116.528368,Hotpot Restaurant
145,Jintai Road Subway Station,39.929399,116.484865,Haidilao Hot Pot (海底捞火锅),39.914400,116.476463,Hotpot Restaurant
153,Jintai Road Subway Station,39.929399,116.484865,羊大爷涮肉,39.921710,116.480619,Hotpot Restaurant
270,TuanJieHu Subway Station,39.939510,116.468363,大得涮肉,39.949578,116.449035,Hotpot Restaurant
293,WangJing Subway Station,40.004785,116.474634,Jubaoyuan Hotpot (聚宝源涮肉),40.003797,116.463959,Hotpot Restaurant
301,WangJing Subway Station,40.004785,116.474634,Haidilao Hot Pot (海底捞火锅),39.988366,116.478230,Hotpot Restaurant
315,WangJing Subway Station,40.004785,116.474634,Haidilao Hot Pot (海底捞火锅),39.995484,116.458247,Hotpot Restaurant
593,Wukesong Subway Station,39.913850,116.280681,Haidilao Hot Pot (海底捞火锅),39.910884,116.297744,Hotpot Restaurant


### Filter all restaurants

In [63]:
beijing_restaurant_venues = beijing_venues[beijing_venues['Venue Category'].str.contains('Restaurant')]
print(beijing_restaurant_venues.shape)
beijing_restaurant_venues.head(20)

(257, 7)


,Subway Station,Subway Latitude,Wubway Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,GuoMao Subway Station,39.914832,116.468148,Din Tai Fung (鼎泰丰),39.909210,116.473768,Dumpling Restaurant
1,GuoMao Subway Station,39.914832,116.468148,Annie's 安妮意大利餐厅,39.912178,116.470545,Italian Restaurant
5,GuoMao Subway Station,39.914832,116.468148,Dadong Roast Duck Restaurant (大董烤鸭店),39.907567,116.473725,Peking Duck Restaurant
6,GuoMao Subway Station,39.914832,116.468148,Haidilao Hot Pot (海底捞火锅),39.914400,116.476463,Hotpot Restaurant
11,GuoMao Subway Station,39.914832,116.468148,Country Kitchen (乡味小厨),39.918469,116.454406,Beijing Restaurant
12,GuoMao Subway Station,39.914832,116.468148,Bistrot B (怡庭),39.918578,116.454910,French Restaurant
16,GuoMao Subway Station,39.914832,116.468148,Hatsune,39.912629,116.453097,Sushi Restaurant
20,GuoMao Subway Station,39.914832,116.468148,Petit Susu (小 Susu),39.910644,116.455449,Vietnamese Restaurant
21,GuoMao Subway Station,39.914832,116.468148,Annie's (安妮意大利餐厅),39.914558,116.449651,Italian Restaurant
22,GuoMao Subway Station,39.914832,116.468148,Dadong Duck (董小味儿),39.911950,116.481404,Peking Duck Restaurant


### 3.5 Analyze Restaurant distribuation around each subway staion

In [52]:
# one hot encoding
beijing_restaurant_onehot = pd.get_dummies(beijing_restaurant_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
beijing_restaurant_onehot['Subway Station'] = beijing_restaurant_venues['Subway Station'] 

# move neighborhood column to the first column
fixed_columns = [beijing_restaurant_onehot.columns[-1]] + list(beijing_restaurant_onehot.columns[:-1])
beijing_restaurant_onehot = beijing_restaurant_onehot[fixed_columns]

print(beijing_restaurant_onehot.shape)
beijing_restaurant_onehot.head(10)

(257, 43)


,Subway Station,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hong Kong Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Peking Duck Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Shanxi Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
0,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,GuoMao Subway Station,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
20,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
21,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,GuoMao Subway Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
beijing_restaurant_grouped = beijing_restaurant_onehot.groupby('Subway Station').mean().reset_index()

print(beijing_restaurant_grouped.shape)
beijing_restaurant_grouped.head(10)

(24, 43)


,Subway Station,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hong Kong Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Peking Duck Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Shanxi Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
0,Caoqiao Subway Station,0.0,0.00,0.0,0.100000,0.100000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.400000,0.000000,0.00,0.000000,0.0,0.100000,0.0,0.00,0.000000,0.0,0.100000,0.100000,0.0,0.000000,0.000000,0.000000,0.10,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.0
1,Changping Subway Station,0.0,0.00,0.0,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.0
2,Changyang Subway Station,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.0
3,Gucheng Subway Station,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.0,0.666667,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.0
4,GuoMao Subway Station,0.0,0.00,0.0,0.052632,0.026316,0.026316,0.184211,0.026316,0.0,0.078947,0.0,0.000000,0.052632,0.00,0.026316,0.0,0.052632,0.0,0.00,0.105263,0.0,0.078947,0.000000,0.0,0.026316,0.052632,0.052632,0.00,0.026316,0.0,0.0,0.0,0.0,0.026316,0.026316,0.000,0.0,0.026316,0.026316,0.026316,0.00,0.0
5,Guozhan Subway Station,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.375000,0.000000,0.0,0.000000,0.0,0.250000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.125,0.0,0.000000,0.000000,0.000000,0.00,0.0
6,Jiangtai Subway Station,0.0,0.08,0.0,0.000000,0.000000,0.040000,0.200000,0.000000,0.0,0.000000,0.0,0.040000,0.000000,0.04,0.000000,0.0,0.000000,0.0,0.04,0.040000,0.0,0.160000,0.000000,0.0,0.000000,0.080000,0.040000,0.04,0.040000,0.0,0.0,0.0,0.0,0.000000,0.040000,0.040,0.0,0.040000,0.000000,0.000000,0.04,0.0
7,Jintai Road Subway Station,0.0,0.00,0.0,0.086957,0.000000,0.000000,0.304348,0.000000,0.0,0.000000,0.0,0.217391,0.000000,0.00,0.000000,0.0,0.086957,0.0,0.00,0.086957,0.0,0.000000,0.043478,0.0,0.000000,0.043478,0.043478,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.043478,0.043478,0.00,0.0
8,Liangxiang Subway Station,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.500000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.0
9,Liangxiang university city Subway Station,0.0,0.00,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.0


In [ ]:
#hood_test='Agincourt'
#temp_test = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood_test].T.reset_index()
#temp_test

In [55]:
num_top_venues = 10

for hood in beijing_restaurant_grouped['Subway Station']:
    print("----"+hood+"----")
    temp = beijing_restaurant_grouped[beijing_restaurant_grouped['Subway Station'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caoqiao Subway Station----
                     venue  freq
0     Fast Food Restaurant   0.4
1       Beijing Restaurant   0.1
2         Ramen Restaurant   0.1
3        Hotpot Restaurant   0.1
4      Japanese Restaurant   0.1
5        Korean Restaurant   0.1
6         Asian Restaurant   0.1
7    Vietnamese Restaurant   0.0
8  New American Restaurant   0.0
9   Peking Duck Restaurant   0.0


----Changping Subway Station----
                             venue  freq
0                 Asian Restaurant   0.5
1               Chinese Restaurant   0.5
2               African Restaurant   0.0
3  Southern / Soul Food Restaurant   0.0
4               Mexican Restaurant   0.0
5          New American Restaurant   0.0
6           Peking Duck Restaurant   0.0
7                 Ramen Restaurant   0.0
8                       Restaurant   0.0
9               Seafood Restaurant   0.0


----Changyang Subway Station----
                       venue  freq
0       Fast Food Restaurant   1.0
1         Afric

### 3.6 Put above data into new dataframe

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Subway Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Subway Station'] = beijing_restaurant_grouped['Subway Station']

for ind in np.arange(beijing_restaurant_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(beijing_restaurant_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Subway Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caoqiao Subway Station,Fast Food Restaurant,Japanese Restaurant,Korean Restaurant,Asian Restaurant,Beijing Restaurant,Hotpot Restaurant,Ramen Restaurant,Zhejiang Restaurant,Hong Kong Restaurant,Hainan Restaurant
1,Changping Subway Station,Asian Restaurant,Chinese Restaurant,Zhejiang Restaurant,Fast Food Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
2,Changyang Subway Station,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant
3,Gucheng Subway Station,Fast Food Restaurant,Chinese Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
4,GuoMao Subway Station,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Dumpling Restaurant,New American Restaurant,Hotpot Restaurant,French Restaurant,Peking Duck Restaurant,Asian Restaurant,Mexican Restaurant
5,Guozhan Subway Station,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Korean Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant
6,Jiangtai Subway Station,Chinese Restaurant,Japanese Restaurant,American Restaurant,New American Restaurant,Restaurant,Ramen Restaurant,Indian Restaurant,Italian Restaurant,Yunnan Restaurant,Fast Food Restaurant
7,Jintai Road Subway Station,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Asian Restaurant,Xinjiang Restaurant,Vietnamese Restaurant,Korean Restaurant,New American Restaurant,Peking Duck Restaurant
8,Liangxiang Subway Station,Japanese Restaurant,Fast Food Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant
9,Liangxiang university city Subway Station,Asian Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant


### 3.7 k-means to cluster the scarborough areas into 5 clusters.

In [121]:
kclusters = 5
beijing_restaurant_grouped_cluster = beijing_restaurant_grouped.drop('Subway Station', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(beijing_restaurant_grouped_cluster)
kmeans.labels_[0:20] 

array([2, 1, 2, 2, 1, 1, 1, 1, 4, 3, 2, 2, 0, 1, 1, 4, 2, 1, 1, 1],
      dtype=int32)

In [122]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

beijing_restaurant_merged = beijing_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
beijing_restaurant_merged = beijing_restaurant_merged.join(neighborhoods_venues_sorted.set_index('Subway Station'), on='Subway Station')
print(beijing_restaurant_merged.shape)
beijing_restaurant_merged.head(10)

(29, 14)


,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,GuoMao Subway Station,116.468148,39.914832,1.0,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Dumpling Restaurant,New American Restaurant,Hotpot Restaurant,French Restaurant,Peking Duck Restaurant,Asian Restaurant,Mexican Restaurant
1,Sihui East Subway Station,116.522091,39.914846,1.0,Fast Food Restaurant,Chinese Restaurant,Korean Restaurant,Hotpot Restaurant,New American Restaurant,Dim Sum Restaurant,Zhejiang Restaurant,Hunan Restaurant,Hong Kong Restaurant,Hainan Restaurant
2,Tongzhou Subway Station,116.672144,39.903694,2.0,Chinese Restaurant,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
3,Jintai Road Subway Station,116.484865,39.929399,1.0,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Asian Restaurant,Xinjiang Restaurant,Vietnamese Restaurant,Korean Restaurant,New American Restaurant,Peking Duck Restaurant
4,TuanJieHu Subway Station,116.468363,39.939510,1.0,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Peking Duck Restaurant,German Restaurant,Spanish Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Asian Restaurant
5,WangJing Subway Station,116.474634,40.004785,1.0,Korean Restaurant,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Japanese Restaurant,Cantonese Restaurant,Peking Duck Restaurant,Hunan Restaurant,Zhejiang Restaurant
6,WangJing East Subway Station,116.493202,40.009192,1.0,Chinese Restaurant,American Restaurant,Korean Restaurant,Cantonese Restaurant,Hong Kong Restaurant,Italian Restaurant,Fast Food Restaurant,Zhejiang Restaurant,French Restaurant,Hunan Restaurant
7,Jiangtai Subway Station,116.496723,39.976800,1.0,Chinese Restaurant,Japanese Restaurant,American Restaurant,New American Restaurant,Restaurant,Ramen Restaurant,Indian Restaurant,Italian Restaurant,Yunnan Restaurant,Fast Food Restaurant
8,Guozhan Subway Station,116.561760,40.075863,1.0,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Korean Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant
9,Lishuiqiao South Subway Station,116.420987,40.048115,2.0,Fast Food Restaurant,Chinese Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant


In [123]:
print(beijing_restaurant_merged.shape)
beijing_restaurant_merged=beijing_restaurant_merged.dropna()
print(beijing_restaurant_merged.shape)
beijing_restaurant_merged.head(10)

(29, 14)
(24, 14)


,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,GuoMao Subway Station,116.468148,39.914832,1.0,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Dumpling Restaurant,New American Restaurant,Hotpot Restaurant,French Restaurant,Peking Duck Restaurant,Asian Restaurant,Mexican Restaurant
1,Sihui East Subway Station,116.522091,39.914846,1.0,Fast Food Restaurant,Chinese Restaurant,Korean Restaurant,Hotpot Restaurant,New American Restaurant,Dim Sum Restaurant,Zhejiang Restaurant,Hunan Restaurant,Hong Kong Restaurant,Hainan Restaurant
2,Tongzhou Subway Station,116.672144,39.903694,2.0,Chinese Restaurant,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
3,Jintai Road Subway Station,116.484865,39.929399,1.0,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Asian Restaurant,Xinjiang Restaurant,Vietnamese Restaurant,Korean Restaurant,New American Restaurant,Peking Duck Restaurant
4,TuanJieHu Subway Station,116.468363,39.939510,1.0,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Peking Duck Restaurant,German Restaurant,Spanish Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Asian Restaurant
5,WangJing Subway Station,116.474634,40.004785,1.0,Korean Restaurant,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Japanese Restaurant,Cantonese Restaurant,Peking Duck Restaurant,Hunan Restaurant,Zhejiang Restaurant
6,WangJing East Subway Station,116.493202,40.009192,1.0,Chinese Restaurant,American Restaurant,Korean Restaurant,Cantonese Restaurant,Hong Kong Restaurant,Italian Restaurant,Fast Food Restaurant,Zhejiang Restaurant,French Restaurant,Hunan Restaurant
7,Jiangtai Subway Station,116.496723,39.976800,1.0,Chinese Restaurant,Japanese Restaurant,American Restaurant,New American Restaurant,Restaurant,Ramen Restaurant,Indian Restaurant,Italian Restaurant,Yunnan Restaurant,Fast Food Restaurant
8,Guozhan Subway Station,116.561760,40.075863,1.0,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Korean Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant
9,Lishuiqiao South Subway Station,116.420987,40.048115,2.0,Fast Food Restaurant,Chinese Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant


In [124]:
beijing_hotpot_venues.head(20)

,Subway Station,Subway Latitude,Wubway Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,GuoMao Subway Station,39.914832,116.468148,Haidilao Hot Pot (海底捞火锅),39.914400,116.476463,Hotpot Restaurant
96,GuoMao Subway Station,39.914832,116.468148,羊大爷涮肉,39.921710,116.480619,Hotpot Restaurant
117,Sihui East Subway Station,39.914846,116.522091,Nanmen Hotpot (南门涮肉),39.915552,116.528368,Hotpot Restaurant
145,Jintai Road Subway Station,39.929399,116.484865,Haidilao Hot Pot (海底捞火锅),39.914400,116.476463,Hotpot Restaurant
153,Jintai Road Subway Station,39.929399,116.484865,羊大爷涮肉,39.921710,116.480619,Hotpot Restaurant
270,TuanJieHu Subway Station,39.939510,116.468363,大得涮肉,39.949578,116.449035,Hotpot Restaurant
293,WangJing Subway Station,40.004785,116.474634,Jubaoyuan Hotpot (聚宝源涮肉),40.003797,116.463959,Hotpot Restaurant
301,WangJing Subway Station,40.004785,116.474634,Haidilao Hot Pot (海底捞火锅),39.988366,116.478230,Hotpot Restaurant
315,WangJing Subway Station,40.004785,116.474634,Haidilao Hot Pot (海底捞火锅),39.995484,116.458247,Hotpot Restaurant
593,Wukesong Subway Station,39.913850,116.280681,Haidilao Hot Pot (海底捞火锅),39.910884,116.297744,Hotpot Restaurant


In [125]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(beijing_restaurant_merged['Latitude'], beijing_restaurant_merged['Longitude'], beijing_restaurant_merged['Subway Station'], beijing_restaurant_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster) - 1],
        fill_opacity=0.7).add_to(map_clusters)

# Add "Hotpot" markers into map
hotpot_cluster = 10
for lat, lon, poi in zip(beijing_hotpot_venues['Venue Latitude'], beijing_hotpot_venues['Venue Longitude'], beijing_hotpot_venues['Subway Station']):
    label = folium.Popup(str(poi) + ' HOTPOT!! ' + str(hotpot_cluster),parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color='darkred',
        fill=True,
        fill_color='darkred',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.8 Cluster Analysis

In [138]:
beijing_restaurant_merged.loc[beijing_restaurant_merged['Cluster Labels'] == 0]

,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Shahe Subway Station,116.294784,40.154767,0.0,Beijing Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant


In [133]:
beijing_restaurant_merged.loc[beijing_restaurant_merged['Cluster Labels'] == 1]

,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,GuoMao Subway Station,116.468148,39.914832,1.0,Chinese Restaurant,Italian Restaurant,Japanese Restaurant,Dumpling Restaurant,New American Restaurant,Hotpot Restaurant,French Restaurant,Peking Duck Restaurant,Asian Restaurant,Mexican Restaurant
1,Sihui East Subway Station,116.522091,39.914846,1.0,Fast Food Restaurant,Chinese Restaurant,Korean Restaurant,Hotpot Restaurant,New American Restaurant,Dim Sum Restaurant,Zhejiang Restaurant,Hunan Restaurant,Hong Kong Restaurant,Hainan Restaurant
3,Jintai Road Subway Station,116.484865,39.929399,1.0,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Asian Restaurant,Xinjiang Restaurant,Vietnamese Restaurant,Korean Restaurant,New American Restaurant,Peking Duck Restaurant
4,TuanJieHu Subway Station,116.468363,39.939510,1.0,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Peking Duck Restaurant,German Restaurant,Spanish Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Asian Restaurant
5,WangJing Subway Station,116.474634,40.004785,1.0,Korean Restaurant,Chinese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Italian Restaurant,Japanese Restaurant,Cantonese Restaurant,Peking Duck Restaurant,Hunan Restaurant,Zhejiang Restaurant
6,WangJing East Subway Station,116.493202,40.009192,1.0,Chinese Restaurant,American Restaurant,Korean Restaurant,Cantonese Restaurant,Hong Kong Restaurant,Italian Restaurant,Fast Food Restaurant,Zhejiang Restaurant,French Restaurant,Hunan Restaurant
7,Jiangtai Subway Station,116.496723,39.976800,1.0,Chinese Restaurant,Japanese Restaurant,American Restaurant,New American Restaurant,Restaurant,Ramen Restaurant,Indian Restaurant,Italian Restaurant,Yunnan Restaurant,Fast Food Restaurant
8,Guozhan Subway Station,116.561760,40.075863,1.0,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Korean Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant
14,Changping Subway Station,116.239879,40.226362,1.0,Asian Restaurant,Chinese Restaurant,Zhejiang Restaurant,Fast Food Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
17,Wuluju Subway Station,116.282962,39.938597,1.0,Hotpot Restaurant,Asian Restaurant,Peking Duck Restaurant,French Restaurant,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hunan Restaurant,Hong Kong Restaurant,Hainan Restaurant


In [134]:
beijing_restaurant_merged.loc[beijing_restaurant_merged['Cluster Labels'] == 2]

,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Tongzhou Subway Station,116.672144,39.903694,2.0,Chinese Restaurant,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
9,Lishuiqiao South Subway Station,116.420987,40.048115,2.0,Fast Food Restaurant,Chinese Restaurant,Szechuan Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
11,Xierqi Subway Station,116.312409,40.059146,2.0,Fast Food Restaurant,Asian Restaurant,Seafood Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
12,Life Science Park Subway Station,116.300677,40.100924,2.0,Chinese Restaurant,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
15,Wukesong Subway Station,116.280681,39.913850,2.0,Fast Food Restaurant,Eastern European Restaurant,Hotpot Restaurant,Thai Restaurant,Chinese Restaurant,Peking Duck Restaurant,Zhejiang Restaurant,Hunan Restaurant,Hong Kong Restaurant,Hainan Restaurant
16,Gucheng Subway Station,116.197048,39.913476,2.0,Fast Food Restaurant,Chinese Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant
21,Changyang Subway Station,116.219290,39.769199,2.0,Fast Food Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant
25,Caoqiao Subway Station,116.358312,39.851701,2.0,Fast Food Restaurant,Japanese Restaurant,Korean Restaurant,Asian Restaurant,Beijing Restaurant,Hotpot Restaurant,Ramen Restaurant,Zhejiang Restaurant,Hong Kong Restaurant,Hainan Restaurant


In [135]:
beijing_restaurant_merged.loc[beijing_restaurant_merged['Cluster Labels'] == 3]

,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Liangxiang university city Subway Station,116.183071,39.729423,3.0,Asian Restaurant,Zhejiang Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant


In [136]:
beijing_restaurant_merged.loc[beijing_restaurant_merged['Cluster Labels'] == 4]

,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Tiantongyuan Subway Station,116.419181,40.081453,4.0,Japanese Restaurant,Fast Food Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant
20,Liangxiang Subway Station,116.163847,39.748704,4.0,Japanese Restaurant,Fast Food Restaurant,Eastern European Restaurant,Indian Restaurant,Hunan Restaurant,Hotpot Restaurant,Hong Kong Restaurant,Hainan Restaurant,German Restaurant,French Restaurant


In [137]:
beijing_restaurant_merged.loc[beijing_restaurant_merged['Cluster Labels'] == 5]

,Subway Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### 4. Observation


#### - Because the output of Foursquare APIs does NOT include so much “hotpot” type data, so I added each found “hotpot” type venue into map with big dark red color circle marker
#### - Most found restaurants data locate in cluster 1 and 2. 
#### - Cluster 1 (purple color) locate in east and northeast of Beijing. It includes few big large commercial and business districts, such as “CBD” and “WangJing”
#### - Cluster 2(light blue) mainly locate in north and northwest of Beijing. It includes few large-scale residential areas, such “Tiantongyuan”, “BeiYuan”,”HuiLongGuan” and so on.
#### - All found “hotpot” data are mainly near with cluster 1.
